In [145]:
from urllib.parse import urlparse
from functools import reduce
import pandas as pd

In [146]:
a = pd.read_csv("1.csv")
b = pd.read_csv("2.csv")
c = pd.read_csv("3.csv")
# drop redundant index column
a.drop(a.columns[[0]], axis=1, inplace=True)
b.drop(b.columns[[0]], axis=1, inplace=True)
c.drop(c.columns[[0]], axis=1, inplace=True)
# print(a.head())
# print(b.head())
# print(c.head())

In [147]:
# In order to find duplicate entires (ngos that were listed on multiple websites)
# we are going to use the URL

def getNetLoc(row):
    website = row['website']
    if isinstance(website, str):
        website = urlparse(website)
        return website[1]
    return ""

In [148]:
a['netloc'] = a.apply(getNetLoc, axis=1)
b['netloc'] = b.apply(getNetLoc, axis=1)
c['netloc'] = c.apply(getNetLoc, axis=1)

In [149]:
# print(a.head())

In [150]:
# Were gonna merge on netloc so if the row doesnt have one remove it to be added later
a2 = a[a['netloc'] != ""]
b2 = b[b['netloc'] != ""]
c2 = c[c['netloc'] != ""]
a3 = a[a['netloc'] == ""]
b3 = b[b['netloc'] == ""]
c3 = c[c['netloc'] == ""]

In [151]:
dfs = [a2, b2, c2]
# The line below should work, but some of the websites that were sourced from had duplicate entries of their own
# causing the validate line to fail. guess it wont be so horrible if my code results in some duplicates ¯\_(ツ)_/¯
# merged = reduce(lambda left,right: pd.merge(left, right, on='netloc', how='outer', validate='many_to_one'), dfs)
merged = reduce(lambda left, right: pd.merge(left, right, on='netloc', how='outer'), dfs)
# print(merged.head())

In [152]:
# Now we are merging in the ngos that didnt have a website, just hoping they arent duplicates
dfs = [merged, a3, b3, c3]

final = reduce(lambda left, right: pd.merge(left, right, how='outer'), dfs)

In [153]:
# output
final.to_csv("merged.csv", index=False)